In [2]:
import mesa
from mesa import Model, Agent
from mesa.datacollection import DataCollector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Users/oyama-kenshin/Desktop/Vault/05_Codes/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [75]:
class Participants(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.has_token = False

        # 資産と効用
        self.wealth = self.model.base_wealth + int(self.random.normalvariate(0, 20))
        self.wealth = max(0, self.wealth)
        # 価値観パラメータ (個性を決定)
        # W_i(1) (独占時の効用) > W_i(N) (飽和時の効用) の制約を満たすように生成
        self.w_1 = self.random.normalvariate(150, 20)
        self.w_n = self.w_1 * self.random.uniform(0.40, 0.60) 

        # 効用が負にならないように調整
        self.w_1 = max(0, self.w_1)
        self.w_n = max(0, self.w_n)

        self.bid_flag = False

        # L_i(N-1) (最大疎外感)
        self.l_0 = 0
        self.l_n_minus_1 = -self.random.normalvariate(225, 40) # 疎外感が小さい
        if self.l_n_minus_1 > 0:
            self.l_n_minus_1 = -self.l_n_minus_1

        #print(f"ResearcherAgent {self.unique_id} initialized with wealth {self.wealth}, w_1 {self.w_1}, w_n {self.w_n}, l_n-1 {self.l_n_minus_1}")
    
    def calculate_bid(self, k, n):
        # W_k+1とL_kの計算
        w_k_plus_1 = self.w_1 - (self.w_1 - self.w_n) * (k / (n - 1))
        l_k = self.l_n_minus_1 * (k / (n - 1))

        #購入額の計算
        bid_price = w_k_plus_1 - l_k
        return bid_price
    
    def calculate_u_diff(self, k):
        """価格を考慮した効用差を計算して返す"""
        n = self.model.num_agents
        price = self.model.initial_price

        net_utility = self.calculate_bid(k,n)
        u_diff = net_utility - price
        return u_diff

    def flag_if_interested(self, k, mid, mad):
        #k=self.model.sold_tokens
        n=self.model.num_agents
        tau = self.model.tau

        if k >= self.model.num_data or self.has_token:
            return False

        #U_buy-U_not_buyの計算
        u_diff = self.calculate_u_diff(k)
        #print(f"Agent {self.unique_id} utility difference: {u_diff}")

        #事前に計算したスケーリングを用いてZスコアを計算
        z = (u_diff - mid) / (mad + 1e-6)  # madが0になるのを防ぐために小さな値を加える

        # ロジットモデルに基づく購入確率の計算
        # tauが0に近いとexpの計算が不安定になるのを防ぐ
        if tau <= 0.01:
            #温度がほぼ0なら、確定的な意思決定と同じ
            prob_buy = 1.0 if z > 0 else 0.0
        else:
            x = -z / tau; prob_buy = 1 / (1 + np.exp(np.clip(x, -60, 60)))

        if self.random.random() < prob_buy and self.model.initial_price <= self.wealth:
            self.bid_flag = True
            return True
        else:
            #print(f"Agent {self.unique_id} did not buy a token")
            return False
        
    def buy_if_flagged(self):
        if self.bid_flag and not self.has_token and self.model.sold_tokens < self.model.num_data:
            # 購入
            self.wealth -= self.model.initial_price
            self.has_token = True
            self.model.sold_tokens += 1
            self.model.provider_revenue += self.model.initial_price
            self.bid_flag = False
            #print(f"Agent {self.unique_id} bought a token")
            return True
        else:
            #print(f"Agent {self.unique_id} did not buy a token")
            self.bid_flag = False
            return False

In [76]:
class DataMarket(Model):
    def __init__(self, num_agents, num_data, initial_price, base_wealth, tau=0.5, seed=None):
        super().__init__(seed=seed)
        self.num_data = num_data
        self.initial_price = initial_price
        self.base_wealth = base_wealth
        self.num_agents = num_agents
        self.tau = tau
         # エージェントの内訳を指定する辞書
        self.steps = 0

        #providerの設定
        self.provider_revenue = 0
        self.sold_tokens = 0

        self.datacollector = DataCollector(
            model_reporters={
                "Holders": lambda m: m.sold_tokens,
                "ProviderRevenue": "provider_revenue"
            }
        )
        self.datacollector.collect(self)

        # エージェントの生成
        self.participants = Participants.create_agents(self, n=num_agents)

    def calculate_scaler(self, u_diffs):
        """効用差のリストを受け取り、効用差の中央値とMADを返す"""
        median = np.median(u_diffs)
        mad = np.median(np.abs(u_diffs - median))
        return median, mad

    def step(self):
        k = self.sold_tokens
        non_holder_agents = self.participants.select(
            lambda agent: not agent.has_token
        )
        U_diffs = [agent.calculate_u_diff(k) for agent in non_holder_agents]
        median, mad = self.calculate_scaler(U_diffs)
        #print(f"Median of U_diffs: {median}, MAD: {mad}")
        non_holder_agents.shuffle_do("flag_if_interested", k=k, mid=median, mad=mad)
        non_holder_agents.shuffle_do("buy_if_flagged")

In [83]:
test_model = DataMarket(num_agents=1000, num_data=1000, initial_price = 200, tau = 1.0, base_wealth=550)

for _ in range(10):
    test_model.step()
    print(f"Step {_+1} complete. Total sold tokens: {test_model.sold_tokens}, Provider revenue: {test_model.provider_revenue}")



Step 1 complete. Total sold tokens: 489, Provider revenue: 97800
Step 2 complete. Total sold tokens: 759, Provider revenue: 151800
Step 3 complete. Total sold tokens: 873, Provider revenue: 174600
Step 4 complete. Total sold tokens: 924, Provider revenue: 184800
Step 5 complete. Total sold tokens: 959, Provider revenue: 191800
Step 6 complete. Total sold tokens: 981, Provider revenue: 196200
Step 7 complete. Total sold tokens: 987, Provider revenue: 197400
Step 8 complete. Total sold tokens: 991, Provider revenue: 198200
Step 9 complete. Total sold tokens: 996, Provider revenue: 199200
Step 10 complete. Total sold tokens: 998, Provider revenue: 199600
